In [3]:
from unsloth import FastLanguageModel
import torch
from transformers import AutoTokenizer

# 4bit 로딩
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 512,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)  # Inference 최적화

==((====))==  Unsloth 2025.5.10: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 4060. Num GPUs = 1. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-15): 16 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear4bit(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear4bit(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear4bit(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear4bit(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear4bit(in_features=6912, out_features=1152, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_l

In [4]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "한화이글스에 대해 알려줘줘"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device).to(torch.bfloat16)


# Gemma-style prompt 자동 처리
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

outputs = model.generate(inputs, max_new_tokens=64, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You have set `compile_config`, but we are unable to meet the criteria for compilation. Compilation will be skipped.


user
You are a helpful assistant.

한화이글스에 대해 알려줘줘
##
##
**한화 이글스**

**한화 이글스**는 2017년 6월 14일에 창단한 한국 야구 리그 (KMB) 소속의 야구팀입니다. 

**개요**

*   **팀 이름


In [13]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import torch

# 1. 모델 로딩 (Unsloth + Gemma 3 1B IT)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# 2. 문서 준비 + chunking (예시)
documents = [
    "한화이글스는 대전광역시를 연고로 하는 한국프로야구 팀입니다.",
    "이 팀은 1986년에 창단되었으며, 대표적인 선수로 정근우, 김태균 등이 있습니다.",
    "한화이글스는 홈구장으로 한화생명이글스파크를 사용하고 있습니다.",
    "한화이글스의 맛집은 농심가락입니다."
]

# 3. 임베딩 모델 및 FAISS 인덱스 준비
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embed_model.encode(documents, convert_to_tensor=True).cpu().detach().numpy()

faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(embeddings)

# 4. 사용자 질문 → 유사 chunk 검색
def retrieve_context(question, top_k=2):
    q_embed = embed_model.encode([question], convert_to_tensor=True).cpu().detach().numpy()
    _, indices = faiss_index.search(q_embed, top_k)
    return "\n".join([documents[i] for i in indices[0]])

# 5. Prompt 구성 + Gemma 추론
def ask_rag(question):
    context = retrieve_context(question)

    messages = [
        {"role": "system", "content": "당신은 친절한 한국어 AI입니다. 질문에 문맥에 맞게 자세히 대답하세요."},
        {"role": "user", "content": f"다음 내용을 참고해서 대답해줘:\n{context}\n\n질문: {question}"}
    ]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
    outputs = model.generate(inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 6. 예시 실행
print(ask_rag("한화이글스의 홈구장은 어디야?"))


==((====))==  Unsloth 2025.5.10: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 4060. Num GPUs = 1. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
user
당신은 친절한 한국어 AI입니다. 질문에 문맥에 맞게 자세히 대답하세요.

다음 내용을 참고해서 대답해줘:
한화이글스의 맛집은 농심가락입니다.
한화이글스는 홈구장으로 한화생명이글스파크를 사용하고 있습니다.

질문: 한화이글스의 홈구장은 어디야?
답변:

한화이글스의 홈구장은 한화생명이글스파크입니다.



In [15]:
# 6. 예시 실행
print(ask_rag("한화이글스의 맛집은 어디야?"))

user
당신은 친절한 한국어 AI입니다. 질문에 문맥에 맞게 자세히 대답하세요.

다음 내용을 참고해서 대답해줘:
한화이글스의 맛집은 농심가락입니다.
한화이글스는 대전광역시를 연고로 하는 한국프로야구 팀입니다.

질문: 한화이글스의 맛집은 어디야?
답변: 한화이글스의 맛집은 농심가락입니다.

**해설:**

이 답변은 주어진 정보만으로는 "농심가락"이 한화이글스의 맛집임을 명확하게 인지하고 있습니다. 

따라서 주어진 정보의 정보를 활용하여,  농심가락이 한화이글스의 맛집으로 이해하고 답변을 제공합니다.

**제외**

*   **다른 맛집 정보:** 다른 맛집 정보(예: '정대리', '어린이수영장' 등)에 대한 질문에는 답변을 제공하지 않습니다.
*   **개인적인 추천:** 개인적인 취향이나 경험을 바탕으로 답변을 제공하지 않습니다.

이해에 도움이 되셨기를 바랍니다.


In [12]:
# 6. 예시 실행
print(ask_rag("한화이글스의 맛집은 어디야?"))

user
당신은 친절한 한국어 AI입니다. 질문에 문맥에 맞게 자세히 대답하세요.

다음 내용을 참고해서 대답해줘:
한화이글스는 대전광역시를 연고로 하는 한국프로야구 팀입니다.
한화이글스는 홈구장으로 한화생명이글스파크를 사용하고 있습니다.

질문: 한화이글스의 맛집은 어디야?
답변: 한화이글스 홈구장인 한화생명이글스파크 내에 다양한 맛집들이 있습니다. 

좀 더 자세하게 설명해 드릴까요? 어떤 종류의 맛집을 찾고 싶으신가요?

*   **식사 종류:** (예: 맛있는 한식, 퓨전 음식, 캐주얼한 식사, 고급 레스토랑)
*   **매장 분위기:** (예: 분위기 좋은 곳, 조용한 곳, 활기찬 곳, 낭만적인 분위기)
*   **가격:** (예: 저렴한 가격, 부담 없는 가격, 비싼 가격)
*   **특징:** (예: 런치 메뉴, 특별한 뷰, 냅킨 수거 정책)
*   **편의시설:** (예: 야외석, 바, 룸)

이러한 요소들을 고려하여 맛집을 추천해 드릴 수 있습니다. 어떤 것을 가장 중요하게 생각하세요?


In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060. Max memory = 7.996 GB.
4.227 GB of memory reserved.


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "mrm8488/t5-base-finetuned-question-generation-ap"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

qg_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

context = "한화이글스는 1986년에 창단되었으며, 대전광역시를 연고로 한다. 홈구장은 한화생명이글스파크이다."

input_text = f"generate question: {context}"
outputs = qg_pipeline(input_text, max_new_tokens=64)
print(outputs[0]['generated_text'])


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can s

question:                               


In [6]:
outputs

[{'generated_text': 'question:                               '}]